# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [21]:
# Importing necessary Python Libraries
import pandas as pd
import numpy as np
import scipy.stats as stats


In [22]:
# Load the dataset and view first five rows
df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [23]:
# check the number of rows/ entries of the DataFrame df
len(df)

8188

In [24]:
# Check columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [25]:
# Load the dataset and view first five rows
df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [26]:
# Examine if there are missing values in the `action` column
df['action'].isna().sum()

0

In [27]:
# Check how many viewers clicked
df['action'].value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [28]:
# Check how many viewers also clicked
clicked = set(df[df.action=='click']['id'].unique())
viewed = set(df[df.action=='view']['id'].unique())
print("Users who viewers: {} \t Users who clicked: {}".format(len(viewed), len(clicked)))
print("Users who viewed but didn't click: {}".format(len(viewed-clicked)))
print("Users who clicked but didn't view: {}".format(len(clicked-viewed)))

Users who viewers: 6328 	 Users who clicked: 1860
Users who viewed but didn't click: 4468
Users who clicked but didn't view: 0


Because all users who clicked the link also viewed the webpage; there is no anomally with the data

In [29]:
# Checking for overlap between the control group and the experiment group
experiment_group = set(df[df.group=='experiment']['id'].unique())
control_group = set(df[df.group=='control']['id'].unique())
print('Overlap between the experiment group and the control group: {}'.format(len(experiment_group & control_group)))

Overlap between the experiment group and the control group: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [30]:
df['count'] = 1
df.head()

,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,1
3,2016-09-24 19:59:02.646620,671993,control,view,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,1


In [31]:
#Convert clicks into a binary variable on a user-by-user-basis
control = df[df.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)



print("Sample sizes:\tControl: {}\tExperiment: {}".format(len(control), len(experiment)))
print("Total Clicks:\tControl: {}\tExperiment: {}".format(control.click.sum(), experiment.click.sum()))
print("Average click rate:\tControl: {}\tExperiment: {}".format(control.click.mean(), experiment.click.mean()))
#control.head()
experiment.head()

Sample sizes:	Control: 3332	Experiment: 2996
Total Clicks:	Control: 932.0	Experiment: 928.0
Average click rate:	Control: 0.2797118847539016	Experiment: 0.3097463284379172


action,click,view
id,,
182988,0.0,1.0
183136,0.0,1.0
183141,1.0,1.0
183283,0.0,1.0
183389,0.0,1.0


In [32]:
def welch_t(a, b):
    
    numerator = a.mean() - b.mean()
    
    # “ddof = Delta Degrees of Freedom”: the divisor used in the calculation is N - ddof, 
    #  where N represents the number of elements. By default ddof is zero.
    
    denominator = np.sqrt(a.var(ddof=1)/a.size + b.var(ddof=1)/b.size)
    
    return np.abs(numerator/denominator)

def welch_df(a, b):
    
    s1 = a.var(ddof=1) 
    s2 = b.var(ddof=1)
    n1 = a.size
    n2 = b.size
    
    numerator = (s1/n1 + s2/n2)**2
    denominator = (s1/ n1)**2/(n1 - 1) + (s2/ n2)**2/(n2 - 1)
    
    return numerator/denominator


def p_value_welch_ttest(a, b, two_sided=False):
  
    t = welch_t(a, b)
    df = welch_df(a, b)
    
    p = 1-stats.t.cdf(np.abs(t), df)
    
    if two_sided:
        return 2*p
    else:
        return p

In [33]:
p_value_welch_ttest(control.click, experiment.click)

0.004466402814337078

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [34]:
#Your code here

# Calculate the click-through rate (CTR) for the control group
control_df = df[df['group'] == 'control']
control_clicks = len(control_df[control_df['action'] == 'click'])
control_views = len(control_df[control_df['action'] == 'view'])
control_ctr = control_clicks / control_views

# Get the number of views in the experiment group
experiment_df = df[df['group'] == 'experiment']
experiment_views = len(experiment_df[experiment_df['action'] == 'view'])

# Calculate the expected number of clicks for the experiment group
expected_experiment_clicks = experiment_views * control_ctr

print(f"Expected clicks for experiment group: {expected_experiment_clicks}")

Expected clicks for experiment group: 838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [35]:
#Your code here

# Get the actual number of clicks in the experiment group
actual_experiment_clicks = len(experiment_df[experiment_df['action'] == 'click'])

# Calculate the variance of clicks in the experiment group, assuming binomial distribution
variance_experiment_clicks = experiment_views * control_ctr * (1 - control_ctr)

# Calculate the standard deviation
std_dev_experiment_clicks = np.sqrt(variance_experiment_clicks)
print(f"std: {std_dev_experiment_clicks}")

# Calculate the z-score
z_score = (actual_experiment_clicks - expected_experiment_clicks) / std_dev_experiment_clicks

print(f"Z-score: {z_score}")

std: 24.568547907005815
Z-score: 3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [36]:
# Your code here

from scipy.stats import norm
# Calculate the p-value for a one-tailed test (since we're testing if the experiment group had *more* clicks)

p_value = 1 - norm.cdf(z_score)  #  Right-tailed p-value

print(f"P-value: {p_value}")

P-value: 0.00012486528006949715


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

Both the p-values returned by the Welch t-test and the one computed using a normal distribuction based on the z-score are lesser than the significance level (alpha = 0.05). Thus the Null Hypothesis that experimental and the control page have a similar impact on homepage viewers who clicked through. Thus, the experimental homepage design is more effective compared to the control

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.